In [1]:
import pandas as pd
data = pd.read_csv('/content/ed-tech_faqs.csv',encoding='cp1252')


In [2]:
data.head()

,prompt,response
0,I have never done programming in my life. Can ...,"Yes, this is the perfect bootcamp for anyone w..."
1,Why should I trust Ed-tech?,Till now 9000 + learners have benefitted from ...
2,Is there any prerequisite for taking this boot...,Our bootcamp is specifically designed for begi...
3,What datasets are used in this bootcamp? Is it...,The datasets used in this bootcamp are crafted...
4,I’m not sure if this bootcamp is good enough f...,We got you covered. Go ahead and watch our you...


In [3]:
pip install langchain gemini-python pandas openai faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.3/195.3 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 13.7 MB/s eta 0:00:00


In [4]:
pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.6 MB/s eta 0:00:00


In [5]:
pip install transformers faiss-cpu

In [6]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.0 MB/s eta 0:00:00


In [7]:
pip install gemini-python langchain

In [8]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from faiss import IndexFlatL2, write_index, read_index

data['context'] = data.apply(lambda row: f"prompt: {row['prompt']}, response: {row['response']}", axis=1)

model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_embeddings(texts):
    embeddings = []
    for text in texts:
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
        with torch.no_grad():
            outputs = model(**inputs)
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
        embeddings.append(embedding)
    return np.array(embeddings)

contexts = data['context'].tolist()
context_embeddings = get_embeddings(contexts)
dimension = context_embeddings.shape[1]
index = IndexFlatL2(dimension)
index.add(context_embeddings)
write_index(index, 'faiss_index.index')
index = read_index('faiss_index.index')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [9]:
import google.generativeai as genai
import os

api_key = '******'
genai.configure(api_key=api_key)

In [10]:
from faiss import IndexFlatL2, read_index
index = read_index('faiss_index.index')

In [11]:
def generate_answer_with_gemini(question, contexts):
    context_text = "\n".join(contexts)
    prompt = f"Given the following context and a question, generate an answer based on this context only. If the answer is not found in the context, kindly state 'I don't know.'\n\nCONTEXT: {context_text}\n\nQUESTION: {question}"
    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(prompt)
    return response.text


In [12]:
def get_relevant_contexts(query, index, tokenizer, model, k=5):
    query_embedding = get_embeddings([query])[0]
    distances, indices = index.search(np.array([query_embedding]), k)
    return [contexts[idx] for idx in indices[0]]

In [13]:
question = "I have never done programming in my life. Can I take this bootcamp?"
relevant_contexts = get_relevant_contexts(question, index, tokenizer, model)
answer = generate_answer_with_gemini(question, relevant_contexts)
print(answer)


Yes, this is the perfect bootcamp for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry. 

